Here I tried to fine-tune a distilbert-base-uncased bert classification model for the 20 most common emojies in sarcastic comments on the twittes dataset. I'm not sure if this is correct.

In [ ]:
! gdown 1sxYXc7gy1ZaYW-L0geDXunEMZ3h9ixSl

Downloading...
From: https://drive.google.com/uc?id=1sxYXc7gy1ZaYW-L0geDXunEMZ3h9ixSl
To: /content/twittes.json
100% 28.7M/28.7M [00:00<00:00, 116MB/s]


In [ ]:
! pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 4.6 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=212a281fdb74f1eb822dadebb95a76cdc8c6c9fa3afa19e2285fe56b4ed2b13d
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
import json

f = open('twittes.json')
twitter_data = json.load(f)
twitter_data = twitter_data['_default']
f.close()

In [ ]:
import re

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def clean_com(com):
  com = com.lower()
  com = re.sub(r'http\S+', '', com)
  com = re.sub("@[A-Za-z0-9_]+","", com)
  com = re.sub("#[A-Za-z0-9_]+","", com)
  com = re.sub("rt :","", com)
  com = re.sub("^ +","", com)
  com = re.sub(" +$","", com)
  com = re.sub("\n","", com)
  com = re.sub(" +"," ", com)
  return remove_emoji(com)

In [ ]:
import emoji
emojis = ['😂','🤣','🙄','🤔','😉','😒','😜','😆','😁','😅','😏','👏','❤️','😎','👍','🙃','🤪','😝','😭','😳']

In [ ]:
tweets = [x['data'] for x in twitter_data.values()]
print(len(tweets))

tweets = [x for x in tweets if len(x['emojis']) == 1]
print(len(tweets))

tweets = [(x['fullText'],x['emojis'][0]) for x in tweets]
tweets = list(dict.fromkeys(tweets))
print(len(tweets))

for i in range(len(tweets)):
  tweets[i] = (clean_com(tweets[i][0]), emoji.emojize(tweets[i][1])) 

tweets = [x for x in tweets if 'sarcasm' not in x[0]]
print(len(tweets))

tweets = [x for x in tweets if 'sarcastic' not in x[0]]
print(len(tweets))

tweets = [x for x in tweets if x[1] in emojis]
print(len(tweets))


99617
71723
60393
59546
59163
36536


In [ ]:
X = []
Y = []
emo_dict = {}
for x,y in tweets:
    X.append(x)
    Y.append(emojis.index(y))
    if y not in emo_dict: emo_dict[y] = [x]
    else: emo_dict[y].append(x)


In [ ]:
for e in emojis:
  print(e,len(emo_dict[e]))

😂 11045
🤣 3067
🙄 7031
🤔 2412
😉 1776
😒 1481
😜 1070
😆 662
😁 901
😅 720
😏 1013
👏 504
❤️ 1174
😎 622
👍 673
🙃 856
🤪 372
😝 294
😭 420
😳 443


In [ ]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 88.6 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 31.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 71.1 MB/s 
     |████████████████████████████████| 1.1 MB 91.1 MB/s 
     |████████████████████████████████| 140 kB 83.4 MB/s 
     |████████████████████████████████| 127 kB 86.9 MB/s 
     |████████████████████████████████| 271 kB 94.1 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |█████████████

In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.8,test_size=0.2,random_state=42)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_test, truncation=True, padding=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
print(len(x_train))
print(len(x_test))

29228
7308


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_test
))

In [ ]:
print(train_dataset)

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(100,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(100,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>


In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=20)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
              epochs=2,
              batch_size=16,
              validation_data=val_dataset.shuffle(100).batch(16))

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/2


AttributeError: ignored